In [17]:
from bs4 import BeautifulSoup as bs
import pymongo
import requests
import pandas as pd
from splinter import Browser

url_1 = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

url_2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

url_3 = 'https://twitter.com/marswxreport?lang=en'

url_4 = 'https://space-facts.com/mars/'

url_5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [18]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.classDB

In [19]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

#response = requests.get(url_1)

/usr/local/bin/chromedriver


In [20]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [21]:
browser.visit(url_1)
html1 = browser.html

In [22]:
soup = bs(html1, 'html.parser')

In [23]:
#print(soup.prettify())

In [24]:
slide_elem = soup.select_one('ul.item_list li.slide')
news_title = slide_elem.find("div", class_='content_title')
slide_elem
news_title.text

'NASA Seeking Partner in Contest to Name Next Mars Rover'

In [25]:
para = soup.find_all('div', class_='article_teaser_body')
para[0].text

'NASA has a class assignment for corporations, nonprofits and educational organizations involved in science and space exploration: partner with the agency to inspire future engineers and scientists by sponsoring a contest to name the next rover to venture to the Red Planet.'

In [28]:
browser.visit(url_2)
browser.click_link_by_partial_text('FULL IMAGE')
html2 = browser.html
soup2 = bs(html2, 'html.parser')

In [29]:
image = soup2.find('img', class_='fancybox-image')

featured_image_url = image['src']
featured_image_url

'/spaceimages/images/mediumsize/PIA20465_ip.jpg'

In [30]:
browser.visit(url_3)
html3 = browser.html

soup3 = bs(html3, 'html.parser')

In [31]:
mars_weather = soup3.find_all('div', class_='js-tweet-text-container')

mars_weather[0].text

'\nJose Morales captured Mars from Chicago last night. 15000 frames for this Mars tonight.  The South Pole, Syrtis Major Planum, and Hellas Planitia are visible.pic.twitter.com/cFkgmdoHDV\n'

In [32]:
tables, = pd.read_html(url_4)
#tables = pd.DataFrame(tables)
tables

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [38]:
facts_html = tables.to_html(header=False,index=False)
facts_html

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [34]:
browser.visit(url_5)
html5 = browser.html

soup5 = bs(html5, 'html.parser')

In [35]:
Mars_hem_title = soup5.find_all('h3')
Mars_hem_title_ls = []
Mars_hem_title_img = []
for hem in Mars_hem_title:
    Mars_hem_title_ls.append(hem.text)
    browser.click_link_by_partial_text(hem.text)
    new_soup = bs(browser.html, 'html.parser')
    mars_img = new_soup.find('img', class_='wide-image')
    Mars_hem_title_img.append(mars_img['src'])
    browser.back()

In [36]:
hemisphere_image_urls = []
for x in range(len(Mars_hem_title_ls)):
    dic = {'title':Mars_hem_title_ls[x], 'img_url':Mars_hem_title_img[x]}
    hemisphere_image_urls.append(dic)
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': '/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': '/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': '/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': '/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]